<a href="https://colab.research.google.com/github/FabioCarlesso/ds-study-codes/blob/main/Codes/pos/fertility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from imblearn.over_sampling import SMOTE
import pandas as pd
from collections import Counter

#1. Abrir o arquivo fertility
dados = pd.read_csv('/content/fertility_Diagnosis.txt')
print(dados.columns)

#2. Segmentar: parte X (atributos); parte Y (classe)
#Segmento dos atributos
dados.atributos = dados.drop(columns=['Output'])
#Segmento da classe
dados.classe = dados['Output']
cont_classe = Counter(dados.classe)
print('Frequencia de classes', cont_classe)

#3. Balancear os dados
oversample = SMOTE() #criar um objeto com o SMOTE de base
X, Y = oversample.fit_resample(dados.atributos,dados.classe)
#o SMOTE recebe as partes originais e retorna equivalentes partes balanceadas

cont_y =Counter(Y)
print('Frequencia de classes após balanceamento', cont_y)

#4.Normalizar os dados

#5.Segmentar em dados para aprendizado e dados para testes
#conjunto de atributos para aprendizado
#conjunto de classes para aprendizados, paralelo ao conjunto de atributos

#conjunto de atributos para testes
#conjunto de classes para testes

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)#Teste baseado em holdout (dois segmentos)


#6.Obter o modelo
from sklearn.ensemble import RandomForestClassifier

rf_fertility = RandomForestClassifier().fit(X_train,Y_train)#Obtem o modelo e cria um objeto [rf_fertility]
#pre teste do modelo
#passar o X_test para o modelo obtido, retornando para um vetor
y_previsto = rf_fertility.predict(X_test)
print('########################')
print('Classes previstas com o X_test')
print(y_previsto)

#Para verificar a acurácia do modelo obtido comparar y_previsto com o Y_test
from sklearn import metrics
print('Acurácia global:', metrics.accuracy_score(Y_test,y_previsto))


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':
/usr/loc

Index(['Season', 'Age', 'Childish_diseases', 'Accident',
       'Surgical_intervention', 'High_fevers', 'alcohol_consumption',
       'Smoking', 'hours_sitting', 'Output'],
      dtype='object')
Frequencia de classes Counter({'N': 88, 'O': 12})
Frequencia de classes após balanceamento Counter({'N': 88, 'O': 88})
########################
Classes previstas com o X_test
['O' 'N' 'N' 'O' 'N' 'O' 'O' 'O' 'N' 'N' 'N' 'O' 'O' 'O' 'O' 'O' 'O' 'N'
 'N' 'O' 'O' 'N' 'O' 'N' 'N' 'N' 'N' 'O' 'O' 'O' 'N' 'O' 'N' 'O' 'O' 'N'
 'N' 'N' 'O' 'O' 'N' 'O' 'O' 'O' 'O' 'N' 'N' 'N' 'N' 'O' 'O' 'N' 'N']
Acurácia global: 0.8867924528301887


In [ ]:
# Validação usando Cross
from sklearn.model_selection import cross_validate
# cross_validate requer modelo obtido, atributos para teste X e Y, numero de segmentos
cv_results = cross_validate(rf_fertility,X_train,Y_train,cv=10, return_train_score=True)
print(sorted(cv_results.keys()))
print(cv_results['test_score'])
print(sorted(cv_results['test_score']))
print(cv_results['train_score'])
print(sorted(cv_results['train_score']))

['fit_time', 'score_time', 'test_score', 'train_score']
[0.92307692 0.92307692 0.84615385 0.91666667 0.83333333 0.75
 0.91666667 0.91666667 0.91666667 1.        ]
[0.75, 0.8333333333333334, 0.8461538461538461, 0.9166666666666666, 0.9166666666666666, 0.9166666666666666, 0.9166666666666666, 0.9230769230769231, 0.9230769230769231, 1.0]
[0.99090909 0.99090909 1.         0.99099099 0.99099099 1.
 0.99099099 0.99099099 0.99099099 0.99099099]
[0.990909090909091, 0.990909090909091, 0.990990990990991, 0.990990990990991, 0.990990990990991, 0.990990990990991, 0.990990990990991, 0.990990990990991, 1.0, 1.0]


In [ ]:
#calcular a média
print('Score médio após cross validation:')
print((cv_results['test_score']).mean())
print((cv_results['train_score']).mean())

Score médio após cross validation
0.8942307692307694
0.9927764127764128


In [ ]:
#6.3 matriz de contingência
from sklearn.metrics import confusion_matrix
#passar para a confusion_matrix:
# Classes reservadas para testes (Y_Test)
# Classes obtidas apos o teste (y_previsto)

matriz_contigencia=confusion_matrix(Y_test,y_previsto, labels=['N','O'])
print(matriz_contigencia)

[[23  4]
 [ 2 24]]


In [ ]:
#6.4 Classification report: demonstrar os indicadores de acurácia do modelo
from sklearn.metrics import classification_report
#parametros: Y_test, y_previsto, nomes das classes
# nomes_classes = ['N', 'O']
nomes_classes = rf_fertility.classes_

class_report = classification_report(Y_test, y_previsto, target_names=rf_fertility.classes_)
print(class_report)

              precision    recall  f1-score   support

           N       0.92      0.85      0.88        27
           O       0.86      0.92      0.89        26

    accuracy                           0.89        53
   macro avg       0.89      0.89      0.89        53
weighted avg       0.89      0.89      0.89        53



In [ ]:
# #7. Salvar o modelo
# #Salvar o modelo obtido em disco
# from pickle import dump
# dump(rf_fertility, open('fertility_randon_forest_model.pkl','wb'))